In [108]:
from IPython.display import JSON
import json
from pyvis.network import Network
import pathlib
import datetime

In [103]:
class Block:
    def __init__(self, text):
        self.text = text
        self.words = self.text.split()
        self.children = []

    def add(self, x):
        if type(x) in [list, tuple]:
            self.children.extend(x)
        else:
            self.children.append(x)
        return x
    
    def traverse(self, callback):
        return [callback(c) for c in self.children]
    
#     def collect(self):
#         collected = Block('')
#         collected.add(self.traverse(lambda a: a))
#         collected = []
#         for c in self.children:
#             print(c.collect())
#             collected.extend([[d.text, c] for a, d in c.collect()])
#         return collected
    
    def markdown(self, level=0):
        prefixes = ['# ', '## ', '', '- ', '', '', '']
        return self.text + '\n' + '\n'.join(prefixes[level]+c.markdown(level+1) for c in self.children)
    
#     def indent(self, n):
#         return ['\t'*n+c for c in self.children]
    
    def tostring(self, l=0):
        return self.text + ''.join(['\n'+('\t'*l)+c.tostring(l+1) for c in self.children])
    
    def find(self, name):
        return list(filter(lambda x: x.text == name, self.children))
    
    def relations(self):
        rlist = []
        for c in self.children:
#             print(c.relations())
            rlist.append([self, c])
#             rlist.extend([[c, t] for t, r in c.relations()])
            rlist.extend(c.relations())
        return rlist
    
    def visualize(self, path='./graph-visualization.html'):
        self.vis = Network(notebook=True, width=800, height=800)
        rel_text = [[' '.join(a.text.split()[:3]) for a in b] for b in self.relations()]
#         print(rel_text)
        for x, y in rel_text:
            for w in [x, y]:
                self.vis.add_node(w)
            self.vis.add_edge(x, y)
        return self.vis.show(path)
    
    def __str__(self):
        return self.tostring()
    
    def __getattr__(self, name):
        nodes = self.find(name)
        if nodes:
            return nodes[0]
        else:
            return self.add(Block(name))
        
def getlinks(node):
    links = []
    for w in node.words:
        if '$' in w:
            links.append(w.split('$'))
    for c in node.children:
        links.extend(getlinks(c))
    return links

In [140]:
# path = 'entities.edma'
# path = '../todo_july.txt'
path = './readme_source.txt'
class Doc:
    def __init__(self, source):
        self.source = source
        with open(self.source, 'r') as source_file:
            self.content = source_file.read()
        self.lines = self.content.splitlines()
        
        self.root = Block('')
        self.levels = {0: self.root}
        for line in self.lines:
            content = line.lstrip()
            if content:
                tabs = len(line) - len(content)
                lblock = Block(content)
                self.levels[tabs+1] = lblock
                self.levels[tabs].add(lblock)
                
    def save(self, file_path):
        with open(file_path, 'w') as savefile:
            savefile.write(str(self.root))
        
    def backup(self):
        backup_dir = 'eparse_backups'
        pathlib.Path(f'./{backup_dir}').mkdir(parents=True, exist_ok=True)
        timestamp = datetime.datetime.now().strftime('%d-%m-%Y_%H-%M-%S')
        backup_path = f'./{backup_dir}/backup_{timestamp}.emda'
        self.save(backup_path)
        return backup_path
                
maindoc = Doc(path)

# print(levels)
# JSON(json.dumps(root, default=vars))

urls = []

for l in getlinks(maindoc.root):
    if l[1] == 'wiki':
        b = f'https://en.wikipedia.org/wiki/{l[0].replace(" ", "_")}'
    else:
        b = l[1]
    urls.append(Block(b))

maindoc.root.Metadata.Links.add(urls)
# maindoc.backup()

# print(root.Metadata.Links)
# print(root)

'07-07-2021_16-08-58'

In [105]:
maindoc.root.visualize()

In [62]:
getlinks(root)

[['serialization', 'wiki'],
 ['YAML', 'https://yaml.org/,'],
 ['OGDL', 'https://ogdl.org/,'],
 ['Python', 'https://www.python.org/']]

In [142]:
print(maindoc.root.markdown())


# EMDA
## Efficient Multipurpose Data Architecture

## https://imgs.xkcd.com/comics/standards.png

## EMDA is a human-readable serialization$wiki language for structured data. It draws inspiration from YAML$https://yaml.org/, OGDL$https://ogdl.org/, and similar languages that minimize extraneous syntax and decoration. The main motivations for creating a new format (ie, the goals the format aims to achieve) are:
Very efficient, painless input and maintenance of SD by humans as a core design feature
- Emphasis on abbreviations, shorthand, and syntactical sugar

Fluid restructuring and reformatting of text and abstract data woven into the lang's workflow

Intuitive interfacing with scripting languages like Python$https://www.python.org/

High flexibility and sensible pathways by which to fit the system to the needs of a particular project

## Overview
emda is a very unrestrictive format (and in some ways borders on a non-format). It embraces the idea that languages should be descriptive 

In [48]:
_

'[{"varName": "content", "varType": "str", "varSize": "62", "varShape": "", "varContent": "extension: md", "isMatrix": false, "isWidget": null}, {"varName": "levels", "varType": "dict", "varSize": "360", "varShape": "7 keys", "varContent": "{0: <__main__.Block object at 0x0000029BE1FB1280>, 1: <__main__.Block object at 0x0000029BE1E2F6A0>, 2: <__main__.Block object at 0x0000029BE1E47DC0>, ...", "isMatrix": false, "isWidget": null}, {"varName": "line", "varType": "str", "varSize": "66", "varShape": "", "varContent": "\\t\\t\\t\\textension: md", "isMatrix": false, "isWidget": null}, {"varName": "lines", "varType": "list", "varSize": "920", "varShape": "101", "varContent": "[\'EMDA\', \'\\\\tEfficient Multipurpose Data Architecture\', \'\\\\thttps://imgs.xkcd.com/comics/standards.png\', \'\\\\tEMDA is a human-readable serialization$wiki  ...", "isMatrix": true, "isWidget": null}, {"varName": "path", "varType": "str", "varSize": "108", "varShape": "", "varContent": "./readme_source.txt", "